# Preprocessing data to graph structure

In [ ]:
%load_ext kedro.extras.extensions.ipython

In [ ]:
%reload_kedro

In [ ]:
from typing import Iterator, Tuple
import re

import pandas as pd
import numpy as np
from kedro.extras.datasets.pandas import CSVDataSet
from kedro.io.core import get_filepath_str

from recommender_gnn.extras.datasets.chunks_dataset import (
 _concat_chunks,
)

pd.options.mode.chained_assignment = None
pd.set_option('expand_frame_repr', True)
pd.set_option("display.max_rows", 999)
pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.expand_frame_repr', False)

In [ ]:
customers = context.catalog.load(customers_path)
articles = context.catalog.load(articles_path)
transactions_train = _concat_chunks(context.catalog.load(transactions_train_path))
transactions_val = _concat_chunks(context.catalog.load(transactions_val_path))

In [ ]:
customers.shape

In [ ]:
articles.shape

In [ ]:
transactions_train.shape

In [ ]:
transactions_val.shape

In [ ]:
def _create_mapping(df: pd.DataFrame, map_column: str):
    """Creates mapping into consecutive integers for given column."""
    ids = df.loc[:, map_column].sort_values().reset_index(drop=True)
    mapping = {v: k for k, v in enumerate(ids)}
    return mapping

In [ ]:
users_mapping = _create_mapping(customers, map_column="customer_id")
items_mapping = _create_mapping(articles, map_column="article_id")


In [ ]:
set(transactions_train.customer_id).issubset(set(users_mapping.keys()))

In [ ]:
transactions_train.replace({"customer_id": users_mapping, "article_id": items_mapping}, inplace=True)

In [ ]:
set(transactions_train.customer_id).issubset(set(users_mapping.values()))

In [ ]:
customers

In [ ]:
transactions_train.sort_values("customer_id")

In [ ]:
customers.loc[customers.customer_id==92343, :]

In [ ]:
customers.shape[0]

In [ ]:
len(transactions.customer_id.unique())

# Preprocessed data check

In [ ]:
dataset = "hm"

In [ ]:
transactions_graph_path = f"{dataset}.transactions_graph"
transactions_mapped_path = f"{dataset}_transactions_mapped"
users_mapping_path = f"{dataset}_users_mapping"
items_mapping_path = f"{dataset}_items_mapping"

In [ ]:
transactions_graph = _concat_chunks(context.catalog.load(transactions_graph_path))
transactions_mapped = _concat_chunks(context.catalog.load(transactions_mapped_path))
users_mapping = context.catalog.load(users_mapping_path)
items_mapping = context.catalog.load(items_mapping_path)

# Saving testing fixtures dataframes

In [ ]:
transactions_graph

In [ ]:
transactions_mapped

In [ ]:
items_mapping

In [ ]:
transactions_concat = transactions_graph.sample(frac=0.01)
print(transactions_concat.shape)

In [ ]:
transactions_concat.to_csv("../src/tests/fixtures/dataframes/bank_concat_transactions.csv", index=False)

In [ ]:
transactions_mapped.loc[:, 'user_id'].value_counts().value_counts()

In [ ]:
transactions_mapped_sample = transactions_mapped.sample(frac=0.01)
print(transactions_mapped_sample.shape)

In [ ]:
transactions_mapped_sample.to_csv("../src/tests/fixtures/dataframes/bank_mapped_transactions.csv", index=False)

In [ ]:
transactions_mapped_sample_small = transactions_mapped.sort_values(by="user_id").iloc[0:100]
transactions_mapped_sample_small

In [ ]:
transactions_mapped_sample_small.to_csv("../src/tests/fixtures/dataframes/bank_mapped_transactions_small.csv", index=False)